In [97]:
#This program implements Fixed-Strategy Iteration CFR
#to a game of liar die

In [106]:
#Define a node
class Node:
    #node definitions
    DOUBT = 0
    ACCEPT = 1
    numActions = 2
    sides = 6 # number of sides of a die
    regretSum = [0.0] * sides
    strategy = [0.0] * sides
    strategySum = [0.0] * sides
    pPlayer = 0.0
    pOpponent = 0.0
    responseNodes = [[Node(numActions)]*(sides+1)]*(sides)
    claimNodes = [[Node(numActions)]*(sides+1)]*(sides)
    u = 0.0 #This field is for holding a utility value for a node
    #Default constructor
    def __init__(self,numActions):
        self.regretSum = [0.0] * numActions
        self.strategy = [0.0] * numActions
        self.strategySum  = [0.0] * numActions
        self.numActions = numActions
        self.pPlayer = 0.0
        self.pOpponent = 0.0
    
    #get liar die node current mixed strategy with regret-matching
    def getStrategy(self):
        normalizingSum = 0.0
        for a in range(len(strategy)):
            strategy[a] = max(regretSum[a],0)
            normalizingSum += strategy[a]
        for a in range(len(strategy)):
            if(normalizingSum > 0):
                strategy[a] /= normalizingSum
            else:
                strategy[a] = 1.0 / len(strategy)
        for a in range(len(strategy)):
            strategySum[a] += pPlayer * strategy[a]
        return strategy
    #get liar die node average mixed strategy
    def getAverageStrategy(self):
        normalizingSum = 0
        for a in range(len(strategySum)):
            normalizingSum += strategySum[a]
        for a in range(len(strategySum)):
            if(normalizingSum > 0):
                strategySum[a] /= normalizingSum
            else:
                strategySum[a] = 1.0/len(strategySum)
        return strategySum

In [107]:
import random
numActions = 2
DOUBT = 0
ACCEPT = 1
numActions = 2
sides = 6 # number of sides of a die
regretSum = [0.0] * sides
strategy = [0.0] * sides
strategySum = [0.0] * sides
pPlayer = 0.0
pOpponent = 0.0
responseNodes = [[Node(numActions)]*(sides+1)]*(sides)

print(len(responseNodes))
print(len(responseNodes[0]))
claimNodes = [[Node(numActions)]*(sides+1)]*(sides)

6
7


DOUBT = 0
ACCEPT = 1
sides = 1
numActions = 2
regretSum = []
strategy = []
strategySum = []
pPlayer = 0.0
pOpponent = 0.0
u = 0.0 #This field is for holding a utility value for a node

def getStrategy():
    normalizingSum = 0.0
    for a in range(len(strategy)):
        strategy[a] = max(regretSum[a],0)
        normalizingSum += strategy[a]
    for a in range(len(strategy)):
        if(normalizingSum > 0):
            strategy[a] /= normalizingSum
        else:
            strategy[a] = 1.0 / len(strategy)
    for a in range(len(strategy)):
        strategySum[a] += pPlayer * strategy[a]
    return strategy

def getAverageStrategy():
    normalizingSum = 0
    for a in range(len(strategySum)):
        normalizingSum += strategySum[a]
    for a in range(len(strategySum)):
        if(normalizingSum > 0):
            strategySum[a] /= normalizingSum
        else:
            strategySum[a] = 1.0/len(strategySum)
    return strategySum



In [100]:
def LiarDieTrainer(sides):
    responseNodes = [[Node(numActions)]*sides]*(sides+1)
    for myClaim in range(sides + 1):
        for oppClaim in range(myClaim + 1,sides + 1):
            responseNodes[myClaim][oppClaim] = Node(
                1 if ((oppClaim == 0) or (oppClaim == 2))  
                else 2)
        claimNodes = [[0.0]*sides]*sides
        for oppClaim in range(sides):
            for roll in range(1,sides + 1):
                claimNodes[oppClaim][roll] = \
                Node(sides - oppClaim)

In [137]:
def train(iterations):
    regret = [0.0] * sides
    rollAfterAcceptingClaim = [0] * sides
    for iter in range(iterations):
        #Initialize rolls with starting probabilities
        for i in range(len(rollAfterAcceptingClaim)):
            rollAfterAcceptingClaim[i] = random.randint(0,5)
        claimNodes[0][rollAfterAcceptingClaim[0]] = Node(numActions)
        claimNodes[0][rollAfterAcceptingClaim[0]].pPlayer = 1
        claimNodes[0][rollAfterAcceptingClaim[0]].pOpponent = 1
        
        #accumulate realization weights forward
        for oppClaim in range(sides + 1):
            #Visit response nodes forward
            if(oppClaim > 0):
                for myClaim in range(oppClaim):
                    node = responseNodes[myClaim][oppClaim]
                    actionProb = node.getStrategy()
                    if (oppClaim < sides):
                        nextNode = claimNodes[oppClaim][rollAfterAcceptingClaim[oppClaim]]
                        nextNode.pPlayer += actionProb[1] * node.pPlayer
                        nextNode.pOpponent += node.pOpponent
            #Visit claim node forward
            if(oppClaim < sides):
                node = claimNodes[oppClaim][rollAfterAcceptingClaim[oppClaim]]
                actionProb = node.getStrategy()
                for myClaim in range(oppClaim+1,sides + 1):
                    nextClaimProb = actionProb[myClaim - oppClaim - 1]
                    if(nextClaimProb > 0):
                        nextNode = responseNodes[oppClaim][myClaim]
                        nextNode.pPlayer += node.pOpponent
                        nextNode.pOpponent += nextClaimProb * node.pPlayer
            
        #Backpropagate utilities, adjust regrets and strategies
        for oppClaim in range(0,sides,-1):
            #visit claim nodes backward
            if (oppClaim < sides):
                node = claimNodes[oppClaim][rollAfterAcceptingClaim[oppClaim]]
                actionProb = node.strategy
                node.u = 0.0
                for myClaim in range(oppClaim+1,sides + 1):
                    actionIndex = myClaim - oppClaim - 1
                    nextNode = responseNodes[oppClaim][myClaim]
                    childUtil = -nextNode.u
                    regret[actionIndex] = childUtil
                    node.u += actionProb[actionIndex] * childUtil
                for a in range(len(actionProb)):
                    regret[a] -= node.u
                    node.regretSum[a] += node.pOpponent * regret[a]
                node.pPlayer = node.pOpponent = 0
            #visit response nodes backward
            if(oppClaim > 0):
                for myClaim in range(oppClaim):
                    node = responseNodes[myClaim][oppClaim]
                    actionProb = node.strategy
                    node.u = 0.0
                    doubtUtil = 1 if (oppClaim > rollAfterAcceptingClaim[myClaim]) else -1
                    regret[DOUBT] = doubtUtil
                    node.u += actionProb[DOUBT] * doubtUtil
                    if(oppClaim < sides):
                        nextNode = claimNodes[oppClaim][rollAfterAcceptingClaim[oppClaim]]
                        regret[ACCEPT] = nextNode.u
                        node.u += actionProb[ACCEPT] * nextNode.u
                    for a in range(len(actionProb)):
                        regret[a] -= node.u
                        node.regretSum[a] += node.pOpponent * regret[a]
                    node.pPlayer = node.pOpponent = 0
            
        #reset strategy sums after half of training
        if(iter == (iterations /2)):
            for nodes in responseNodes:
                for node in nodes:
                    if(node != None):
                        for a in range(len(node.strategySum)):
                            node.strategySum[a] = 0
            for nodes in claimNodes:
                for node in nodes:
                    if(node != None):
                        for a in range(len(node.strategySum)):
                            node.strategySum[a] = 0
                        
                    
    #Print strategy
    for initialRoll in range(1,sides+1):
        print('Initial claim policy with role {0}'.format(initialRoll))
        for prob in claimNodes[0][initialRoll].getAverageStrategy():
            print('{0:2.2f}'.format(initialRoll))
    print('\nOld Claim\tNew Claim\tActionProbabilities')
    for myClaim in range(sides+1):
        for oppClaim in range(myClaim + 1, sides+1):
            print('\t{0}\t{1}\t\t{2}'.format(myClaim,oppClaim,responseNodes[myClaim][oppClaim].getAverageStrategy()))
    print('\nOld Claim\tRoll\tActionProbabilities')
    for oppClaim in range(sides):
        for roll in range(1,sides+1):
            print('{0}\t\t{1}\t{2}'.format(oppClaim,roll,str(claimNodes[oppClaim][roll].getAverageStrategy())))


In [138]:
train(1000000)

Initial claim policy with role 1
1.00
1.00
1.00
1.00
1.00
1.00
Initial claim policy with role 2
2.00
2.00
2.00
2.00
2.00
2.00
Initial claim policy with role 3
3.00
3.00
3.00
3.00
3.00
3.00
Initial claim policy with role 4
4.00
4.00
4.00
4.00
4.00
4.00
Initial claim policy with role 5
5.00
5.00
5.00
5.00
5.00
5.00
Initial claim policy with role 6
6.00
6.00
6.00
6.00
6.00
6.00

Old Claim	New Claim	ActionProbabilities
	0	1		[0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669]
	0	2		[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
	0	3		[0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669, 0.16666666666666669]
	0	4		[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
	0	5		[0.16666666666666669, 0.16666666666666669, 